In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

#img1 = cv.imread('Small_area_rotated.png', cv.IMREAD_GRAYSCALE)
img1 = cv.imread('Small_area.png', cv.IMREAD_GRAYSCALE)
img2 = cv.imread('StarMap.png', cv.IMREAD_GRAYSCALE)

img1 = cv.GaussianBlur(img1,(5,5),cv.BORDER_DEFAULT)
img2 = cv.GaussianBlur(img2,(5,5),cv.BORDER_DEFAULT)

sift = cv.SIFT_create(50000)

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

#-- Matching descriptor vectors with a FLANN based matcher
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(des1, des2, 2)
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.8
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
                   
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img1, kp1, img2, kp2, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

#-- Show detected matches
cv.imshow('Good Matches & Object detection', img_matches)
cv.waitKey(0)
cv.destroyAllWindows()     

In [2]:
#-- Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)
for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = kp1[good_matches[i].queryIdx].pt[0]
    obj[i,1] = kp1[good_matches[i].queryIdx].pt[1]
    scene[i,0] = kp2[good_matches[i].trainIdx].pt[0]
    scene[i,1] = kp2[good_matches[i].trainIdx].pt[1]
H, _ =  cv.findHomography(obj, scene, cv.RANSAC)
#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img1.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img1.shape[1]
obj_corners[2,0,1] = img1.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img1.shape[0]
scene_corners = cv.perspectiveTransform(obj_corners, H)
#-- Draw lines between the corners (the mapped object in the scene - image_2 )
cv.line(img_matches, (int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1])), (int(scene_corners[1,0,0] + img1.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[1,0,0] + img1.shape[1]), int(scene_corners[1,0,1])), (int(scene_corners[2,0,0] + img1.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[2,0,0] + img1.shape[1]), int(scene_corners[2,0,1])), (int(scene_corners[3,0,0] + img1.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[3,0,0] + img1.shape[1]), int(scene_corners[3,0,1])), (int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)


cv.imwrite("result.png",img_matches)

#-- Show detected matches
cv.imshow('Good Matches & Object detection', img_matches)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
[ (int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1])),  (int(scene_corners[1,0,0] + img1.shape[1]), int(scene_corners[1,0,1]))]

[(969, 150), (1082, 150)]

In [4]:
[(int(scene_corners[1,0,0] + img1.shape[1]), int(scene_corners[1,0,1])), (int(scene_corners[2,0,0] + img1.shape[1]), int(scene_corners[2,0,1]))]

[(1082, 150), (1083, 263)]

In [5]:
[(int(scene_corners[2,0,0] + img1.shape[1]), int(scene_corners[2,0,1])), (int(scene_corners[3,0,0] + img1.shape[1]), int(scene_corners[3,0,1]))]

[(1083, 263), (968, 264)]

In [6]:
[ (int(scene_corners[3,0,0] + img1.shape[1]), int(scene_corners[3,0,1])), (int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1]))]

[(968, 264), (969, 150)]

In [7]:
(int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1]))

(969, 150)

In [8]:
result_points = [
    (int(scene_corners[0,0,0]), int(scene_corners[0,0,1])),
    (int(scene_corners[1,0,0]), int(scene_corners[1,0,1])),
    (int(scene_corners[3,0,0]), int(scene_corners[3,0,1])),
    (int(scene_corners[0,0,0]), int(scene_corners[0,0,1]))
]

In [9]:
result_points

[(855, 150), (968, 150), (854, 264), (855, 150)]

In [10]:
from solution import Solution

img1 = cv.imread('Small_area.png')
img2 = cv.imread('StarMap.png')

img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)

img1_copy = cv.cvtColor(img1, cv.COLOR_RGB2GRAY)
img2_copy = cv.cvtColor(img2, cv.COLOR_RGB2GRAY)

solution = Solution()
points = solution.localize_object(img1_copy, img2_copy)

print(points)

image_detected = np.copy(img2)

cv.line(image_detected, points[0], points[1], (0,255,0), 4)
cv.line(image_detected, points[1], points[2], (0,255,0), 4)
cv.line(image_detected, points[2], points[3], (0,255,0), 4)
cv.line(image_detected, points[3], points[0], (0,255,0), 4)


#-- Show detected matches
cv.imshow('Good Matches & Object detection', image_detected)
cv.waitKey(0)
cv.destroyAllWindows()

Solution initialized...
[(855, 150), (968, 150), (969, 263), (855, 264)]
